<a href="https://colab.research.google.com/github/darpan02-cypher/Knowledge-Data-and-Discovery/blob/main/HW7_Himanshi_TopicModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install gensim

In [33]:
!pip install -U numpy==1.23.5

  Using cached numpy-1.23.5.tar.gz (10.7 MB)
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [34]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [35]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Specify the path to your folder in Google Drive
folder_path = '/content/drive/MyDrive/KDD/MovieReviews' # Replace with the actual path to your folder

# List files in the folder
try:
    files = os.listdir(folder_path)
    print("Files found the folder:")

except FileNotFoundError:
    print(f"Error: Folder not found at {folder_path}")
except Exception as e:
    print(f"An error occurred: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Files found the folder:


In [36]:
import os

import pandas as pd


file_path = '/content/drive/MyDrive/KDD/MovieReviews2'
documents = []


try:
    files = os.listdir(folder_path)
    print(f"Files found in the folder: {len(files)}")

    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r', encoding='latin-1') as f:
            content = f.read()
            documents.append(content)

    print(f"Successfully loaded {len(documents)} documents.")

except FileNotFoundError:
    print(f"Error: Folder not found at {folder_path}")
except Exception as e:
    print(f"An error occurred: {e}")

Files found in the folder: 180
Successfully loaded 180 documents.


# Task
Load and preprocess the corpus data from "/content/drive/MyDrive/KDD/MovieReviews 2.csv" into the Python session, making the data ready for topic modeling.

## Load the data

### Subtask:
Load the data from the specified file path into a pandas DataFrame.


In [37]:
df = pd.DataFrame(documents, columns=['review'])
print("DataFrame Head:")
display(df.head())
print("\nDataFrame Info:")
df.info()

DataFrame Head:


,review
0,"For twelve years, Johnny Dingle (Andrew Lowery..."
1,"Starring: Mark Dacascos, Paco Christian Prieto..."
2,"Starring: Mandy Patinkin, James Spader, M. Emm..."
3,"Starring: Jason James Richter, Lori Petty, Mic..."
4,"In the quiet New England town of Castle Rock, ..."



DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  180 non-null    object
dtypes: object(1)
memory usage: 1.5+ KB


## Clean the text

### Subtask:
Preprocess the text data by removing punctuation, numbers, and converting text to lowercase.


**Reasoning**:
Define a function to clean the text by converting to lowercase, removing punctuation, and removing numbers, then apply it to the 'review' column to create a 'cleaned_review' column.



In [38]:
import re
import string

def clean_text(text):
    """Converts text to lowercase, removes punctuation and numbers."""
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    return text

df['cleaned_review'] = df['review'].apply(clean_text)

print("DataFrame with cleaned reviews:")
display(df.head())

DataFrame with cleaned reviews:


,review,cleaned_review
0,"For twelve years, Johnny Dingle (Andrew Lowery...",for twelve years johnny dingle andrew lowery h...
1,"Starring: Mark Dacascos, Paco Christian Prieto...",starring mark dacascos paco christian prieto s...
2,"Starring: Mandy Patinkin, James Spader, M. Emm...",starring mandy patinkin james spader m emmet w...
3,"Starring: Jason James Richter, Lori Petty, Mic...",starring jason james richter lori petty michae...
4,"In the quiet New England town of Castle Rock, ...",in the quiet new england town of castle rock a...


## Tokenize the text

### Subtask:
Split the text into individual words or tokens.


**Reasoning**:
Tokenize the cleaned reviews using NLTK's word_tokenize and store the tokens in a new column.



In [39]:
import nltk
from nltk.tokenize import word_tokenize

# Download the 'punkt' tokenizer data
nltk.download('punkt_tab')

df['tokens'] = df['cleaned_review'].apply(word_tokenize)

print("DataFrame with tokens:")
display(df.head())

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


DataFrame with tokens:


,review,cleaned_review,tokens
0,"For twelve years, Johnny Dingle (Andrew Lowery...",for twelve years johnny dingle andrew lowery h...,"[for, twelve, years, johnny, dingle, andrew, l..."
1,"Starring: Mark Dacascos, Paco Christian Prieto...",starring mark dacascos paco christian prieto s...,"[starring, mark, dacascos, paco, christian, pr..."
2,"Starring: Mandy Patinkin, James Spader, M. Emm...",starring mandy patinkin james spader m emmet w...,"[starring, mandy, patinkin, james, spader, m, ..."
3,"Starring: Jason James Richter, Lori Petty, Mic...",starring jason james richter lori petty michae...,"[starring, jason, james, richter, lori, petty,..."
4,"In the quiet New England town of Castle Rock, ...",in the quiet new england town of castle rock a...,"[in, the, quiet, new, england, town, of, castl..."


## Remove stop words

### Subtask:
Eliminate common English stop words from the tokenized text.


**Reasoning**:
Import necessary NLTK modules and download the stopwords corpus if not already present, then create a set of English stopwords and apply a function to remove them from the 'tokens' column of the DataFrame.



In [40]:
import nltk
from nltk.corpus import stopwords

try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def remove_stopwords(tokens):
    """Removes English stop words from a list of tokens."""
    return [word for word in tokens if word not in stop_words]

df['tokens_without_stopwords'] = df['tokens'].apply(remove_stopwords)

print("DataFrame with tokens after removing stop words:")
display(df.head())

DataFrame with tokens after removing stop words:


,review,cleaned_review,tokens,tokens_without_stopwords
0,"For twelve years, Johnny Dingle (Andrew Lowery...",for twelve years johnny dingle andrew lowery h...,"[for, twelve, years, johnny, dingle, andrew, l...","[twelve, years, johnny, dingle, andrew, lowery..."
1,"Starring: Mark Dacascos, Paco Christian Prieto...",starring mark dacascos paco christian prieto s...,"[starring, mark, dacascos, paco, christian, pr...","[starring, mark, dacascos, paco, christian, pr..."
2,"Starring: Mandy Patinkin, James Spader, M. Emm...",starring mandy patinkin james spader m emmet w...,"[starring, mandy, patinkin, james, spader, m, ...","[starring, mandy, patinkin, james, spader, emm..."
3,"Starring: Jason James Richter, Lori Petty, Mic...",starring jason james richter lori petty michae...,"[starring, jason, james, richter, lori, petty,...","[starring, jason, james, richter, lori, petty,..."
4,"In the quiet New England town of Castle Rock, ...",in the quiet new england town of castle rock a...,"[in, the, quiet, new, england, town, of, castl...","[quiet, new, england, town, castle, rock, unas..."


## Lemmatize the text

### Subtask:
Reduce words to their base or dictionary form.


**Reasoning**:
Import necessary libraries, download wordnet if needed, instantiate the lemmatizer, define a function for lemmatization, apply it to the DataFrame, and display the result.



In [41]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

try:
    wordnet.ensure_loaded()
except LookupError:
    nltk.download('wordnet')
    nltk.download('omw-1.4') # Often needed with wordnet

lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    """Lemmatizes a list of tokens."""
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens

df['lemmatized_tokens'] = df['tokens_without_stopwords'].apply(lemmatize_tokens)

print("DataFrame with lemmatized tokens:")
display(df.head())

DataFrame with lemmatized tokens:


,review,cleaned_review,tokens,tokens_without_stopwords,lemmatized_tokens
0,"For twelve years, Johnny Dingle (Andrew Lowery...",for twelve years johnny dingle andrew lowery h...,"[for, twelve, years, johnny, dingle, andrew, l...","[twelve, years, johnny, dingle, andrew, lowery...","[twelve, year, johnny, dingle, andrew, lowery,..."
1,"Starring: Mark Dacascos, Paco Christian Prieto...",starring mark dacascos paco christian prieto s...,"[starring, mark, dacascos, paco, christian, pr...","[starring, mark, dacascos, paco, christian, pr...","[starring, mark, dacascos, paco, christian, pr..."
2,"Starring: Mandy Patinkin, James Spader, M. Emm...",starring mandy patinkin james spader m emmet w...,"[starring, mandy, patinkin, james, spader, m, ...","[starring, mandy, patinkin, james, spader, emm...","[starring, mandy, patinkin, james, spader, emm..."
3,"Starring: Jason James Richter, Lori Petty, Mic...",starring jason james richter lori petty michae...,"[starring, jason, james, richter, lori, petty,...","[starring, jason, james, richter, lori, petty,...","[starring, jason, james, richter, lori, petty,..."
4,"In the quiet New England town of Castle Rock, ...",in the quiet new england town of castle rock a...,"[in, the, quiet, new, england, town, of, castl...","[quiet, new, england, town, castle, rock, unas...","[quiet, new, england, town, castle, rock, unas..."


## Prepare for topic modeling

### Subtask:
Create a dictionary and a corpus (bag-of-words representation) from the preprocessed text, which are required formats for many topic modeling algorithms.


**Reasoning**:
Create a Gensim dictionary and corpus from the lemmatized tokens and display their information.



In [42]:
from gensim.corpora import Dictionary

# Create a Gensim Dictionary from the lemmatized tokens
dictionary = Dictionary(df['lemmatized_tokens'])

# Create a Gensim Corpus (bag-of-words representation)
corpus = [dictionary.doc2bow(tokens) for tokens in df['lemmatized_tokens']]

# Print information about the dictionary and corpus
print("Gensim Dictionary and Corpus created.")
print(f"Number of unique tokens in the dictionary: {len(dictionary)}")
print(f"Number of documents in the corpus: {len(corpus)}")
print("\nFirst 5 documents in the corpus (bag-of-words representation):")
for i, doc in enumerate(corpus[:5]):
    print(f"Document {i+1}: {doc}")


Gensim Dictionary and Corpus created.
Number of unique tokens in the dictionary: 14234
Number of documents in the corpus: 180

First 5 documents in the corpus (bag-of-words representation):
Document 1: [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 8), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 6), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 2), (41, 2), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 2), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2), (64, 1), (65, 1), (66, 1), (67, 4), (68, 1), (69, 2), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 2), (81, 1), (82, 1), (83, 1), (84, 5), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1



### Data Analysis Key Findings

*   The initial attempt to load the data from the specified CSV path failed because the file was not located there. The data was successfully loaded into a DataFrame from an existing Python variable named `documents`, which contained 180 movie reviews.
*   The text data was successfully cleaned by converting it to lowercase and removing punctuation and numbers, resulting in a new column `cleaned_review`.
*   The cleaned text was successfully tokenized into individual words and stored in the `tokens` column after downloading the necessary NLTK 'punkt' and 'punkt_tab' data.
*   Common English stop words were removed from the tokens, creating the `tokens_without_stopwords` column.
*   The tokens were lemmatized to their base form using the WordNet Lemmatizer after downloading the 'wordnet' and 'omw-1.4' NLTK data, and the results were stored in the `lemmatized_tokens` column.
*   A Gensim Dictionary was created from the lemmatized tokens, containing 14234 unique tokens.
*   A Gensim Corpus (bag-of-words representation) was created for the 180 documents, which is a format suitable for topic modeling algorithms.

### Insights or Next Steps

*   The preprocessed data, including the Gensim Dictionary and Corpus, is now ready to be used as input for training a topic modeling algorithm such as Latent Dirichlet Allocation (LDA).
*   Further analysis could involve exploring the most frequent terms in the dictionary or visualizing the corpus structure.


#Correctly apply the LDA topic modeling function and get the correct topic modeling result (topic IDs and their top 10 words).


In [43]:
from gensim.models import LdaModel

# Set the number of topics
num_topics = 5  # we can adjust this number as needed

# Train the LDA model
# 'corpus' and 'dictionary' were created in previous steps
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)


# Print the topics and their top words
print(f"LDA Model with {num_topics} topics:")
for topic_id, topic_words in lda_model.print_topics(num_words=10):
    print(f"Topic {topic_id}: {topic_words}")

LDA Model with 5 topics:
Topic 0: 0.010*"film" + 0.006*"movie" + 0.004*"one" + 0.003*"whale" + 0.003*"time" + 0.003*"life" + 0.003*"story" + 0.002*"much" + 0.002*"scene" + 0.002*"never"
Topic 1: 0.015*"film" + 0.007*"one" + 0.007*"movie" + 0.004*"story" + 0.004*"time" + 0.004*"character" + 0.004*"get" + 0.003*"see" + 0.003*"even" + 0.003*"life"
Topic 2: 0.014*"film" + 0.007*"one" + 0.007*"movie" + 0.005*"life" + 0.005*"like" + 0.004*"story" + 0.004*"get" + 0.003*"character" + 0.003*"make" + 0.003*"even"
Topic 3: 0.016*"film" + 0.008*"one" + 0.005*"even" + 0.005*"story" + 0.004*"get" + 0.004*"like" + 0.003*"see" + 0.003*"take" + 0.003*"make" + 0.003*"movie"
Topic 4: 0.014*"film" + 0.005*"one" + 0.005*"movie" + 0.005*"story" + 0.004*"get" + 0.004*"life" + 0.004*"make" + 0.003*"time" + 0.003*"father" + 0.003*"like"


In [44]:
# Print the topics
print(topic_words)

0.014*"film" + 0.005*"one" + 0.005*"movie" + 0.005*"story" + 0.004*"get" + 0.004*"life" + 0.004*"make" + 0.003*"time" + 0.003*"father" + 0.003*"like"


#Correctly apply the print topics function for all the documents in the corpus and get the correct result (topic coverage scores for each document in the corpus).


In [45]:
# Get topic coverage scores for each document
print("Topic coverage scores for each document:")
for i, doc_topics in enumerate(lda_model[corpus]):
    print(f"Document {i+1}: {doc_topics}")

Topic coverage scores for each document:
Document 1: [(2, 0.99770993)]
Document 2: [(2, 0.9983725)]
Document 3: [(4, 0.99800324)]
Document 4: [(1, 0.9985967)]
Document 5: [(2, 0.99820596)]
Document 6: [(1, 0.99806434)]
Document 7: [(4, 0.99826944)]
Document 8: [(4, 0.9977199)]
Document 9: [(1, 0.99800164)]
Document 10: [(0, 0.9988982)]
Document 11: [(1, 0.99856377)]
Document 12: [(1, 0.4518651), (3, 0.54707026)]
Document 13: [(1, 0.99771994)]
Document 14: [(1, 0.9981115)]
Document 15: [(4, 0.99795526)]
Document 16: [(2, 0.9979596)]
Document 17: [(2, 0.9975562)]
Document 18: [(2, 0.9984969)]
Document 19: [(1, 0.99754065)]
Document 20: [(1, 0.9982981)]
Document 21: [(2, 0.9975706)]
Document 22: [(1, 0.11497803), (2, 0.883679)]
Document 23: [(2, 0.9975109)]
Document 24: [(2, 0.9972014)]
Document 25: [(1, 0.9980641)]
Document 26: [(1, 0.997879)]
Document 27: [(2, 0.99772006)]
Document 28: [(4, 0.9974053)]
Document 29: [(1, 0.2117443), (4, 0.78653795)]
Document 30: [(1, 0.24535789), (3, 0.7